#### Creating CNN with Functional API

In [1]:
import numpy as np
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [2]:
(x_train,y_train),(x_test,y_test) = mnist.load_data()

In [3]:
num_labels = len(np.unique(y_train))
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [4]:
image_size = x_train.shape[1]

x_train = np.reshape(x_train,[-1, image_size, image_size, 1])
x_test = np.reshape(x_test,[-1, image_size, image_size, 1])

In [5]:
x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255

In [6]:
input_shape = (image_size,image_size,1)
batch_size = 128
kernel_size = 3
filters = 64
dropout = 0.3

In [8]:
inputs = Input(shape=input_shape)
y = Conv2D(filters=filters,
          kernel_size = kernel_size,
          activation='relu')(inputs)
y = MaxPooling2D()(y)
y = Conv2D(filters=filters,
          kernel_size=kernel_size,
          activation='relu')(y)
y = MaxPooling2D()(y)
y = Conv2D(filters=filters,
         kernel_size=kernel_size,
         activation='relu')(y)
y = Flatten()(y)
y = Dropout(dropout)(y)
outputs = Dense(num_labels,activation='softmax')(y)

In [9]:
model = Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 64)        36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0     

In [10]:
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics='accuracy')

In [11]:
model.fit(x_train,y_train,
         validation_data=(x_test,y_test),
         epochs=15,
         batch_size = batch_size)

Epoch 1/15
469/469 [==============================] - 13s 28ms/step - loss: 1.3062 - accuracy: 0.7218 - val_loss: 0.2726 - val_accuracy: 0.9624
Epoch 2/15
469/469 [==============================] - 13s 28ms/step - loss: 0.5657 - accuracy: 0.9380 - val_loss: 0.1322 - val_accuracy: 0.9798
Epoch 3/15
469/469 [==============================] - 13s 28ms/step - loss: 0.3413 - accuracy: 0.9594 - val_loss: 0.1375 - val_accuracy: 0.9810
Epoch 4/15
469/469 [==============================] - 13s 28ms/step - loss: 0.2301 - accuracy: 0.9691 - val_loss: 0.0934 - val_accuracy: 0.9840
Epoch 5/15
469/469 [==============================] - 13s 28ms/step - loss: 0.2077 - accuracy: 0.9717 - val_loss: 0.1236 - val_accuracy: 0.9836
Epoch 6/15
469/469 [==============================] - 13s 28ms/step - loss: 0.1786 - accuracy: 0.9738 - val_loss: 0.1185 - val_accuracy: 0.9841
Epoch 7/15
469/469 [==============================] - 13s 29ms/step - loss: 0.1827 - accuracy: 0.9750 - val_loss: 0.0892 - val_accuracy:

In [12]:
score = model.evaluate(x_test,y_test,batch_size = batch_size,verbose = 2)

79/79 - 1s - loss: 0.1842 - accuracy: 0.9902


In [13]:
print("\nTest accuracy: %.1f%%"%(100.0*score[1]))


Test accuracy: 99.0%


#### 2-input and 1-output model (Y-model with Functional API)

In [49]:
from tensorflow.keras.layers import concatenate

In [50]:
from tensorflow.keras.utils import plot_model

In [51]:
(x_train,y_train),(x_test,y_test) = mnist.load_data()

num_labels = len(np.unique(y_train))
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

image_size = x_train.shape[1]

x_train = np.reshape(x_train,[-1, image_size, image_size, 1])
x_test = np.reshape(x_test,[-1, image_size, image_size, 1])

x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255

In [57]:
input_shape = (image_size,image_size,1)
batch_size = 128
kernel_size = 3
dropout = 0.4
n_filters = 32

In [58]:
#left branch
left_inputs = Input(shape = input_shape)
x = left_inputs
filters = n_filters

# 3 - layers of Conv-Dropout-Maxpool
for i in range(3):
    x = Conv2D(filters=filters,
              kernel_size = kernel_size,
              padding='same',
              activation = 'relu')(x)
    x = Dropout(dropout)(x)
    x = MaxPooling2D()(x)
    filters *= 2

#=========================================================
    
#right branch
right_inputs = Input(shape=input_shape)
y = right_inputs

# 3 - layers of Conv-Dropout-Maxpool
for i in range(3):
    y = Conv2D(filters=filters,
              kernel_size = kernel_size,
              padding='same',
              activation = 'relu',
              dilation_rate=2)(y)
    
    y = Dropout(dropout)(y)
    y = MaxPooling2D()(y)
    filters *= 2



In [59]:
import os
y = concatenate([x,y])

y=Flatten()(y)
y=Dropout(dropout)(y)
outputs = Dense(num_labels,activation='softmax')(y)

model = Model([left_inputs,right_inputs],outputs)
# plot_model(model,show_shapes=True)
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_22 (InputLayer)           [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
input_23 (InputLayer)           [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_55 (Conv2D)              (None, 28, 28, 32)   320         input_22[0][0]                   
__________________________________________________________________________________________________
conv2d_58 (Conv2D)              (None, 28, 28, 256)  2560        input_23[0][0]                   
____________________________________________________________________________________________

In [60]:
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [62]:
model.fit([x_train,x_train],y_train,
          validation_data=([x_test,x_test],y_test),
         epochs= 1,
         batch_size = batch_size)

469/469 [==============================] - 526s 1s/step - loss: 0.0732 - accuracy: 0.9784 - val_loss: 0.0836 - val_accuracy: 0.9867


In [63]:
score = model.evaluate([x_test,x_test],
                      y_test,
                      batch_size=batch_size,
                      verbose=2)
print("\nTest accuracy: %.1f%%"%(100.0*score[1]))

79/79 - 29s - loss: 0.0836 - accuracy: 0.9867

Test accuracy: 98.7%
